In [1]:
using Symbolics
using LinearAlgebra
using SHA
using Statistics

In [6]:
# cnt=0
# f=open("signature.txt","r")
# for lines in readlines(f)
#     cnt=cnt+count(i->(i=='x'), lines)
# end
# close(f)
# println(cnt)


# generate sparse polynomials

In [2]:
@variables x[1:64]

1-element Vector{Symbolics.Arr{Num, 1}}:
 x[1:64]

In [3]:
function modify_coeffcients(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        new_coef=value%q
        B+=new_coef*key
    end
    return B
end

modify_coeffcients (generic function with 1 method)

In [4]:
#t monomials
#bound the degree of mononials by b, 1-b inclusive
#coefficients go from 1 to q-1(inclusive)
function sparse_polynomial(n,t,b,q)
    @variables x[1:n]
    
    sparse_poly=0
    
    for i in 1:t #we want t monomials
        deg=rand(1:b)
        
        monomial_d=1
        for j in 1:deg
            monomial_d*=x[rand(1:n)]
        end
        sparse_poly+=rand(1:(q-1))*monomial_d
        
    end
#     println(sparse_poly)
    A=modify_coeffcients(sparse_poly,q)
    return A
end

A=sparse_polynomial(3,5,3,6)
A

2x[3] + 3(x[2]^2)*x[1] + 2(x[3]^2)*x[2]

# Generate kl matrix

In [5]:
function num_identity(k)
    a=zeros(Num,k)
    a[1]=1
    for i in 2:k
        b=zeros(Num,k)
        b[i]=1
        a=[a b] 
    end
    return a
end
# I_mat=num_identity(4)
# I_mat[1,1]=x[2]*x[3]+x[1]
# I_mat


num_identity (generic function with 1 method)

In [74]:
function generate_U(k,n,t,B,q)
    Es=[]
    for i in 1:(k-1)
        for j in (i+1):k
            I_matrix=num_identity(k)
            I_matrix[i,j]=sparse_polynomial(n,t,B,q)
            push!(Es,I_matrix)
        end
    end
#     I_matrix=num_identity(k)#generate a k by k identity matrix
#     for i in 1:Int((k*k-k)/2)
#     b=rand(2:k)
#     a=rand(1:(b-1))
#     I_matrix[a,b]=sparse_polynomial(n,t,B,q)
#     end

    return Es
end 
U=generate_U(5,32,3,3,6)


res_U=U[1]
for i in 2:Int(size(U)[1])
    res_U=res_U*U[i]
end
res_U
# simplify.(expand.(res_U))
# simplify.(expand.(res_U*expand.(inv(res_U))))


5×5 Matrix{Num}:
 1  …  (3x[23] + 3x[6]*x[25] + 2x[3]*x[4]*x[28])*(4x[21] + x[13]*x[22]*x[24] + 3x[5]*x[16]*x[27]) + (x[15]*x[30] + 4x[2]*x[3]*x[30] + 5x[6]*x[13]*x[32])*(3x[7] + (2x[2] + 3x[20]*x[22] + 4x[8]*x[16]*x[27])*((4x[19] + 5x[11]*x[12]*x[18] + x[22])*(4x[21] + x[13]*x[22]*x[24] + 3x[5]*x[16]*x[27]) + 5(x[22]^2)*x[8] + 2x[2]*x[18]*x[24] + x[24]) + (2x[6]*x[31] + x[11]*x[18]*x[29] + 4x[14]*x[27]*x[30])*(4x[21] + x[13]*x[22]*x[24] + 3x[5]*x[16]*x[27]) + 2x[6]*x[19] + 4x[10]*x[23]*x[25]) + (3x[5]*x[11] + 5x[17]*x[20] + 5x[1]*x[19]*x[21])*((4x[19] + 5x[11]*x[12]*x[18] + x[22])*(4x[21] + x[13]*x[22]*x[24] + 3x[5]*x[16]*x[27]) + 5(x[22]^2)*x[8] + 2x[2]*x[18]*x[24] + x[24]) + x[1]*x[13] + 4x[6]*x[31] + 2x[8]*x[16]*x[27]
 0                                                                                                                                                                                                                                                                          

In [514]:
# U[1]*U[2]*U[3]

3×3 Matrix{Num}:
 1  …  (x[22]*x[28] + x[1]*x[8]*x[27] + 3x[18]*x[21])*(3x[15]*x[29] + 5x[28]*x[30] + x[22]) + 2x[8] + x[3]*x[8]*x[13] + 4x[21]*x[31]
 0                                                                                      x[22]*x[28] + x[1]*x[8]*x[27] + 3x[18]*x[21]
 0                                                                                                                                 1

In [75]:
uSize=Int(size(U)[1])
invU=inv(U[uSize])
for i in 1:(uSize-1)
    invU=invU*inv(U[uSize-i])
end
invU

5×5 Matrix{Num}:
 1.0  …        -x[1]*x[13] - 4x[6]*x[31] - 2x[8]*x[16]*x[27]
 0.0                -3x[23] - 3x[6]*x[25] - 2x[3]*x[4]*x[28]
 0.0         -3x[5]*x[11] - 5x[17]*x[20] - 5x[1]*x[19]*x[21]
 0.0     -x[15]*x[30] - 4x[2]*x[3]*x[30] - 5x[6]*x[13]*x[32]
 0.0                                                       1.0

In [76]:
simplify.(expand.(invU*res_U))

5×5 Matrix{Num}:
 1.0  0    0    0    0
 0.0  1.0  0    0    0
 0.0  0.0  1.0  0    0
 0.0  0.0  0.0  1.0  0
 0.0  0.0  0.0  0.0  1.0

In [77]:
res_U=modify_matrix_coef(res_U,6)
invU=modify_matrix_coef(invU,6)
modify_matrix_coef(res_U*invU,6)

5×5 Matrix{Num}:
 1.0  0    0    0    0
 0.0  1.0  0    0    0
 0.0  0.0  1.0  0    0
 0.0  0.0  0.0  1.0  0
 0.0  0.0  0.0  0.0  1.0

In [79]:
function generate_L(k,n,t,B,q)
    Es=[]
    for i in 2:k
        for j in 1:(i-1)
            I_matrix=num_identity(k)
            I_matrix[i,j]=sparse_polynomial(n,t,B,q)
            push!(Es,I_matrix)
        end
    end
#     I_matrix=num_identity(k)#generate a k by k identity matrix
#     for i in 1:Int((k*k-k)/2)
#         a=rand(2:k)
#         b=rand(1:a-1)
#         I_matrix[a,b]=sparse_polynomial(n,t,B,q)
#     end

    return Es
end

L=generate_L(5,32,3,3,6)

res_L=L[1]
for i in 2:Int(size(L)[1])
    res_L=res_L*L[i]
end
res_L
# simplify.(expand.(inv(L)))


5×5 Matrix{Num}:
                                       1  …  0
            3x[11] + 5x[17]*x[31] + x[3]     0
 2x[2]*x[7] + 2x[14]*x[24]*x[29] + x[25]     0
         2x[13] + x[14]^2 + 5x[11]*x[27]     0
    4x[18] + 4x[25] + 3x[13]*x[21]*x[22]     1

In [520]:
# L[1]*L[2]*L[3]

3×3 Matrix{Num}:
                                                     1  …                             0  0
 2x[11]*x[30] + x[14]*x[29]*x[44] + 5x[26]*x[31]*x[36]                                1  0
                   4x[38] + 2x[31]*x[44]*x[50] + x[13]     3x[1] + 5x[36] + 2x[2]*x[31]  1

In [80]:
lSize=Int(size(L)[1])
invL=inv(L[lSize])
for i in 1:(lSize-1)
    invL=invL*inv(L[lSize-i])
end
invL

5×5 Matrix{Num}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      1.0  …  0.0
                                                                                                                                                                                                                                                                                                                                     

In [81]:
simplify.(expand.(invL*res_L))

5×5 Matrix{Num}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0    0.0  1.0  0.0  0.0
 0    0    0.0  1.0  0.0
 0    0    0    0    1.0

In [82]:
res_L=modify_matrix_coef(res_L,6)
invL=modify_matrix_coef(invL,6)
modify_matrix_coef(res_L*invL,6)

5×5 Matrix{Num}:
 1.0  0.0  0.0  0.0  0.0
 0    1.0  0.0  0.0  0.0
 0    0    1.0  0.0  0.0
 0    0    0    1.0  0.0
 0    0    0    0    1.0

2×2 Matrix{Num}:
 2(x[1]^2)*(x[13]^2)*x[7] + 3x[23]*x[25]*x[41] + 3x[23]*x[37]*x[42] + (x[13]^2)*x[1]*x[37]*x[42] + 3(x[13]^2)*x[1]*x[25]*x[41] + 3(x[13]^2)*x[25]*x[31]*x[41] + 3(x[13]^2)*x[31]*x[37]*x[42]  …  3x[23] + 3(x[13]^2)*x[31] + 5(x[13]^2)*x[1]
                                                                                                                                                    3x[25]*x[41] + 4x[1]*x[7] + 5x[37]*x[42]                                               1

In [592]:
# res_UL=modify_matrix_coef(res_U,6)*modify_matrix_coef(res_L,6)
# res_UL=modify_matrix_coef(res_UL,6)
# res_inv=modify_matrix_coef(inv(res_L),6)*modify_matrix_coef(inv(res_U),6)
# res_inv=modify_matrix_coef(res_inv,6)

res_UL=res_U*res_L
res_UL=modify_matrix_coef(res_UL,6)

res_inv=inv(res_L)*inv(res_U)
res_inv=modify_matrix_coef(res_inv,6)
modify_matrix_coef(res_UL*res_inv,6)

3×3 Matrix{Num}:
                                                    0  …  x[5]*x[9]*x[10] + 2.0x[9]*x[32] + 2.0(x[5]^2)*x[10]*x[11] + 4.0x[5]*x[11]*x[32] + x[5]*x[10]*x[18]*x[22] + 2.0x[9]*x[13]*x[27] + 2.0x[18]*x[22]*x[32] + 2.0x[13]*x[18]*x[22]*x[27] + 4.0x[5]*x[11]*x[13]*x[27]
 3x[16]*x[21] + 3x[4]*x[15]*x[17] + 3x[4]*x[23]*x[30]                       3.0x[10]*x[16]*x[21] + 3.0x[4]*x[10]*x[15]*x[17] + 3.0(x[32]^2)*x[16]*x[21]*x[23] + 3.0(x[23]^2)*(x[32]^2)*x[4]*x[30] + 3.0x[4]*x[10]*x[23]*x[30] + 3.0(x[32]^2)*x[4]*x[15]*x[17]*x[23]
                                                    0                       0.0

In [578]:
# res_UL=res_U*res_L
# res_inv=inv(res_L)*inv(res_U)
# simplify.(expand.(res_UL*res_inv))


2×2 Matrix{Num}:
 1  0
 0  1.0

In [593]:
#mod does not work for res_UL and res_inv
# res_UL=modify_matrix_coef(res_UL,6)
# res_inv=modify_matrix_coef(res_inv,6)
# modify_matrix_coef(res_UL*res_inv,6)

res_UL=substitute.(res_UL, (substitution_dict,))
for i in 1:size(res_UL,1)
    for j in 1:size(res_UL,2)
        res_UL[i,j]=mod(res_UL[i,j],6)
    end
end
res_inv=substitute.(res_inv, (substitution_dict,))
for i in 1:size(res_inv,1)
    for j in 1:size(res_inv,2)
        res_inv[i,j]=mod(res_inv[i,j],6)
    end
end
res_I=res_UL*res_inv
res_I

3×3 Matrix{Num}:
 0.0   8.0   4.0
 0.0  16.0  12.0
 0.0   4.0   6.0

In [545]:
res_inv=modify_matrix_coef(res_inv,6)


2×2 Matrix{Num}:
                                1.0  …  3x[13] + 4x[10]*x[13] + 2x[15]*x[16]*x[23]
 3x[8] + 3(x[28]^2)*x[16] + x[19]                                                3.0x[8]*x[13] + 3.0x[13]*x[19] + 3.0(x[28]^2)*x[13]*x[16] + 4.0x[10]*x[13]*x[19] + 2.0x[15]*x[16]*x[19]*x[23]

2×2 Matrix{Num}:
 0  0.0
 0  0.0

In [8]:
function cols_to_keep(k,l)
    col_opts=[i for i in 1:k]
    for i in 1:(k-l)
        random_number=size(col_opts,1)
        splice!(col_opts, rand(1:random_number))
    end
    return col_opts
end
# cols_to_keep(4,3)

cols_to_keep (generic function with 1 method)

In [9]:
function random_del_cols(k,l,matrix)
#     col_opts=[i for i in 1:l]
    col_opts=cols_to_keep(k,l)
    matrix_kl=zeros(Num, k,l)
    
    j=1
    for i in col_opts
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    end
    return col_opts,matrix_kl
end

random_del_cols (generic function with 1 method)

In [10]:
function find_kl_inverse(to_keep,S_inv)
    k=size(S_inv,1)#find the number of rows
    ml_inv=zeros(Num, length(to_keep),k)
    
    j=1
    for i in to_keep
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    end
    return ml_inv
end

find_kl_inverse (generic function with 1 method)

In [11]:
function modify_neg_coeff(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients) 
        new_coef=mod(value,q)
        B+=new_coef*key
    end
    B
end

function modify_matrix_coef(matrix,q)
    res=simplify.(expand.(matrix))#expand and then simplify
    
    for i in 1:size(res,1)
        for j in 1:size(res,2)
            a=res[i,j]

            if isone(a)==false && iszero(a)==false            
                res[i,j]=modify_neg_coeff(a,q)
            end
        end
    end
    return res
end

modify_matrix_coef (generic function with 1 method)

In [12]:
function random_permutation(k)
    cols=[i for i in 1:k]
    ncols=[]
    for i in 1:k
        random_number=size(cols,1)
        a=rand(1:random_number)
        append!(ncols,cols[a])
        splice!(cols, a)
    end
    A=zeros(Num,k,k)
    
    for i in 1:k
        A[i,ncols[i]]=1
    end
    return A
end
    
# P=random_permutation(4)
# inv(P)*P


random_permutation (generic function with 1 method)

In [13]:
# function matrix_multiplication(A,B)#make sure A and B are of appropriate size
#     nrows=size(A)[1]
#     ncols=size(B)[2]
# #     println(nrows)
# #     println(ncols)
#     resM=zeros(Num,nrows,ncols)
#     for i in 1:nrows
#         for j in 1:ncols
#             arr1=A[i,:]
#             arr2=B[:,j]
#             element=0
#             for k in 1:length(arr1)
#                 element=element+arr1[k]*arr2[k]
#             end
#             resM[i,j]=simplify(expand(element))
#             end
#     end
#     return resM
# end


matrix_multiplication (generic function with 1 method)

In [63]:
# A=num_identity(3)
# A[1,1]=x[1]^2+x[2]^2
# A[1,2]=x[2]+x[3]

# B=num_identity(3)
# B[1,3]=x[3]+x[5]^2
# B[2,1]=x[1]
# B

# simplify(expand(A*B))
# matrix_multiplication(D,C)

In [123]:
function matrix_kl_mod(k,l,n,t,B,q)
   
    coin_toss=rand(0:1)
    if coin_toss==2
        U=generate_U(k,n,t,B,q)
        res_U=U[1]
        for i in 2:Int(size(U)[1])
            res_U=res_U*U[i]
        end

        uSize=Int(size(U)[1])
        invU=inv(U[uSize])
        for i in 1:(uSize-1)
            invU=invU*inv(U[uSize-i])
        end

        println(simplify.(expand.(res_U*invU)))
        println("----------------------------")
        res_U=modify_matrix_coef(res_U,q)
        invU=modify_matrix_coef(invU,q)  
        println(modify_matrix_coef(res_U*invU,q))

        col_keep,kl_matrix=random_del_cols(k,l,res_U)
        kl_inverse=find_kl_inverse(col_keep,invU)
        println(modify_matrix_coef(kl_inverse*kl_matrix,q))
        println("----------------------------")
        return res_U,invU,kl_matrix,kl_inverse
    else
        L=generate_L(k,n,t,B,q)
        res_L=L[1]
        for i in 2:Int(size(L)[1])
            res_L=res_L*L[i]
        end
        
        lSize=Int(size(L)[1])
        invL=inv(L[lSize])
        for i in 1:(lSize-1)
            invL=invL*inv(L[lSize-i])
        end
        println(simplify.(expand.(res_L*invL)))
        println("----------------------------")
        res_L=modify_matrix_coef(res_L,q)
        invL=modify_matrix_coef(invL,q)
        println(modify_matrix_coef(res_L*invL,q))
        println("----------------------------")
        
        col_keep,kl_matrix=random_del_cols(k,l,res_L)
        kl_inverse=find_kl_inverse(col_keep,invL)
        println(modify_matrix_coef(kl_inverse*kl_matrix,q))
        println("----------------------------")
        return res_L,invL,kl_matrix,kl_inverse
    end
    


    
end

matrix_kl_mod (generic function with 2 methods)

In [135]:
A,B,C,D=matrix_kl_mod(6,3,50,3,3,6)

Num[1.0 0.0 0.0 0.0 0.0 0.0; 0 1.0 0.0 0.0 0.0 0.0; 0 0 1.0 0.0 0.0 0.0; 0 0 0 1.0 0.0 0.0; 0 0 0 0 1.0 0.0; 0 0 0 0 0 1.0]
----------------------------
Num[1.0 0.0 0.0 0.0 0.0 0.0; 0 1.0 0.0 0.0 0.0 0.0; 0 x[3] + x[4] + x[5] 1.0 0.0 0.0 0.0; 0 0 0 1.0 0.0 0.0; 0 0 0 0 1.0 0.0; 0 0 0 0 0 1.0]
----------------------------
Num[1.0 0.0 0.0; 0 1.0 0.0; 0 0 1.0]
----------------------------


(Num[1 0 … 0 0; 4x[22] + 2x[42] + 2x[10]*x[18]*x[48] 1 … 0 0; … ; 2x[9]*x[30] + 3x[29]*x[36] + 5x[20]*x[26] 3x[20] + 4x[7]*x[22] + 4x[11]*x[26]*x[40] … 1 0; 5x[5] + 2x[13] + 2x[8]*x[20]*x[25] 4(x[19]^2)*x[8] + 2x[1]*x[22] + 4x[23]*x[43] … 3x[4]*x[42] + 2x[40]*x[49] + 5x[28]*x[48]*x[50] 1], Num[1.0 0.0 … 0.0 0.0; 2x[22] + 4x[42] + 4x[10]*x[18]*x[48] 1.0 … 0.0 0.0; … ; x[20]*x[26] + 4(x[22]^2)*x[7] + 4x[9]*x[30] + 3x[29]*x[36] + 2x[3]*x[22]*x[32] + 2x[5]*x[22]*x[35] + 2x[5]*x[32]*x[42] + 4x[3]*x[22]*x[35] + 4x[3]*x[32]*x[42] + 4x[4]*x[22]*x[35] + 4x[4]*x[32]*x[42] + x[18]*x[25]*x[26]*x[36] + 2x[7]*x[22]*x[42] + 4x[5]*x[22]*x[32] + 2x[3]*x[35]*x[42] + 2x[4]*x[22]*x[32] + 2x[4]*x[35]*x[42] + 4x[5]*x[35]*x[42] + x[1]*x[25]*x[36]*x[45] + 2x[25]*x[26]*x[30]*x[36] + 4x[11]*x[22]*x[26]*x[40] + 3x[11]*x[27]*x[29]*x[45] + 2x[11]*x[26]*x[40]*x[42] + 3x[3]*x[12]*x[18]*x[26] + 2x[27]*x[29]*x[33]*x[35] + 4(x[45]^2)*x[1]*x[19]*x[43] + 3x[5]*x[11]*x[33]*x[34] + 3x[11]*x[27]*x[29]*x[33] + 2x[27]*x[29]*x

In [130]:
modify_matrix_coef(D*C,6)

5×5 Matrix{Num}:
 1.0  0.0  0.0  0.0  0.0
 0    1.0  0.0  0.0  0.0
 0    0    1.0  0.0  0.0
 0    0    0    1.0  0.0
 0    0    0    0    1.0

In [278]:
# function matrix_kl(k,l,s,n,t,B,q)
#     Ls=[]
#     Us=[]
#     for i in 1:s
#         U=generate_U(k,n,t,B,q)
#         L=generate_L(k,n,t,B,q)
#         push!(Us,U)
#         push!(Ls,L)
#     end 
#     U_new=Us[rand(1:s)]
#     L_new=Ls[rand(1:s)]
    
#     P1=random_permutation(k)
#     P2=random_permutation(k)
#     kk_matrix=matrix_multiplication(matrix_multiplication(matrix_multiplication(U_new,P1),L_new),P2)
# #     kk_matrix=simplify.(expand.(kk_matrix))
# #     kk_matrix=modify_matrix_coef(kk_matrix,q)#modify the coefficients for kk_matrix
#     kk_inv=inv(P2)*inv(L_new)*inv(P1)*inv(U_new)#able to compute, not able to verify the result
# #     kk_inv=simplify.(expand.(kk_inv))
#     col_keep,kl_matrix=random_del_cols(k,l,kk_matrix)
#     kl_inverse=find_kl_inverse(col_keep,kk_inv)
  
#     return kk_matrix,kk_inv,kl_matrix, kl_inverse
# end 

# substitution_dict = Dict()
# for i in 1:64
#     substitution_dict[x[i]] = rand(0:1)
# end

function matrix_kl(k,l,n,t,B,q)
    #generate upper unitriangular matrix
    U=generate_U(k,n,t,B,q)
    res_U=U[1]
    for i in 2:Int(size(U)[1])
        res_U=res_U*U[i]
    end

    #generate lower unitriangular matrix
    L=generate_L(k,n,t,B,q)
    res_L=L[1]
    for i in 2:Int(size(L)[1])
        res_L=res_L*L[i]
    end

    X=res_U*res_L

    
    uSize=Int(size(U)[1])
    invU=inv(U[uSize])
    for i in 1:(uSize-1)
        invU=invU*(inv(U[uSize-i]))
    end

    
    lSize=Int(size(L)[1])
    invL=inv(L[lSize])
    for i in 1:(lSize-1)
        invL=invL*(inv(L[lSize-i]))
    end


#     println(simplify.(expand.(invU*res_U)))
#     println(simplify.(expand.(invL*res_L)))
    Y=(invL)*(invU)
#     println(simplify.(expand.(X*Y)))

    
    col_keep,kl_matrix=random_del_cols(k,l,X)
    kl_inverse=find_kl_inverse(col_keep,Y)
    return X,Y,kl_matrix,kl_inverse
end

matrix_kl (generic function with 1 method)

In [285]:
# A,B,C,D=matrix_kl(4,4,50,3,3,6)

A,B,C,D=matrix_kl(5,3,64,2,3,6)
println(size(C))
print(size(D))

(5, 3)
(3, 5)

In [286]:
simplify.(expand.(D*C))

3×3 Matrix{Num}:
 1.0  0    0
 0    1.0  0
 0    0    1.0

In [19]:
# prod_AB=A*B
# prod_inv=inv(A*B)
# simplify.(expand.(prod_AB*prod_inv))
# prod_AB=modify_matrix_coef(prod_AB,6)
# prod_inv=modify_matrix_coef(prod_inv,6)
# prod_AB*prod_inv

4×4 Matrix{Num}:
 1    0    0    0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [278]:
# substitution_dict = Dict()
# for i in 1:64
#     substitution_dict[x[i]] = rand(0:1)
# end
# substitute.(modify_matrix_coef(simplify.(expand.(D*C)),6), (substitution_dict,))


3×3 Matrix{Num}:
  0.0  0    0.0
 -2.0  0.0  0.0
  0    0    0

In [194]:

file=open("publicKey.txt","w")
for i in 1:Int(size(C,1))
    for j in 1:Int(size(C,2))
        write(file,string(C[i,j]))
        write(file," ")
    end
    write(file,"\n")
end
close(file)
filesize("publicKey.txt")

12384

In [195]:
file=open("privateKey.txt","w")
for i in 1:Int(size(D,1))
    for j in 1:Int(size(D,2))
        write(file,string(D[i,j]))
        write(file," ")
    end
    write(file,"\n")
end
close(file)
filesize("privateKey.txt")

16853

In [52]:
I66=zeros(Num,3,3)
for i in 1:Int(3*3)
    a=rand(1:3)
    b=rand(1:3)
    I66[a,b]=sparse_polynomial(32,3,3,6)
end
I66

3×3 Matrix{Num}:
            4x[31] + 4x[15]*x[32] + 5x[1]*x[23]  …                   3x[20] + 3x[29] + 4x[6]*x[31]
 5x[2]*x[18] + 4x[6]*x[24] + 4x[10]*x[16]*x[32]     2x[8]*x[26] + 3x[4]*x[21] + 5x[18]*x[22]*x[31]
          2x[7] + 2x[7]*x[8] + x[8]*x[11]*x[17]       2x[2]*x[4] + 4x[12]*x[31] + 3x[6]*x[8]*x[24]

In [53]:
simplify.(expand.(I66*inv(I66)))

LoadError: InterruptException:

# hashing 

In [272]:
message = "digital sig"
@variables x[1:64]
function hashing512(message)
    hash = sha2_512(message)
    bitstring512=""
    for i in hash
        bitstring512=bitstring512*bitstring(i)
    end
    
    A512=[]
    for i in bitstring512
        append!(A512,Int(i)-48)
    end
    return A512
end

hashing512 (generic function with 1 method)

In [287]:
function polys(message)
    coeffs=[]
    A512=hashing512(message)
    last12=A512[501:512,]
    for i in 0:3
        b3=last12[(1+3*i):3*(i+1),]
        coef=0
        for j in 1:3
            coef=coef+2^(3-j)*b3[j]
        end
        coef=mod(coef,6)
        append!(coeffs,coef)
    end
#     println(coeffs)
    
    first300=A512[1:300]
    vars=[]
    for i in 1:50
        v=first300[(1+6*(i-1)):(6*i),]
        vn=0
        for j in 1:6
            vn=vn+2^(6-j)*v[j]
        end
        vn=vn+1
        append!(vars,vn)
    end
#     println(vars)
    
    middle200=A512[301:500]
    mons=[]
    for i in 1:5
        b40=middle200[(1+40*(i-1)):(40*i)]

        for j in 1:4
            b10=b40[(1+10*(j-1)):(10*j)]
            mon=1

            for k in 1:10
                if b10[k]==1 

                    if vars[10*(j-1)+k]!=0
                        mon=mon*x[vars[10*(j-1)+k]]
                    end
                end
            end
            append!(mons,mon)

        end
    end
#     println(mons)
    
    polys=[]
    for i in 1:5
        pol=0
        for j in 1:4
            pol=pol+mons[4*(i-1)+j]*coeffs[j]
        end
        append!(polys,pol)
    end
 
    
    Un=zeros(Num,1,3)
    for i in 1:3
        Un[1,i]=polys[i]
    end
    
#     Un=simplify.(expand.(Un))
    U=zeros(Num,3,1)
    for i in 1:3
        U[i,1]=polys[i]
    end
    return U,Un
end

U,Un=polys(message)
println(size(U))
println(size(Un))


(3, 1)
(1, 3)


In [288]:
V=Un*D
sizeof(V)


40

In [275]:
V

1×5 Matrix{Num}:
 (4x[3]*x[31]*x[34] + 3x[16]*x[24]*x[25]*x[30]*x[44]*x[58])*((-x[30]*x[63] - 3x[4]*x[18]*x[38])*(-x[39] - 5x[25]*x[33]*x[35]) - 5x[6]*x[27]*x[47] - 2x[10]*x[13]*x[27]) + (3x[16]*x[25]*x[35]*x[43]*x[44]*x[58] + 4x[3]*x[9]*x[20]*x[29]*x[31]*x[34]*x[43]*x[52]*x[57])*((-5x[6]*x[27]*x[47] - 2x[10]*x[13]*x[27])*((-x[1]*x[45] - 5x[19]*x[49])*(-4x[37]*x[58] - 4x[24]*x[31]*x[62]) - 4x[10] - x[24]*x[54]) + (-x[4]*x[14] - 3x[12]*x[47]*x[57])*(-4x[37]*x[58] - 4x[24]*x[31]*x[62]) + (-x[30]*x[63] - 3x[4]*x[18]*x[38])*((-x[39] - 5x[25]*x[33]*x[35])*((-x[1]*x[45] - 5x[19]*x[49])*(-4x[37]*x[58] - 4x[24]*x[31]*x[62]) - 4x[10] - x[24]*x[54]) + (-x[26]*x[46] - 2x[22]*x[49])*(-4x[37]*x[58] - 4x[24]*x[31]*x[62]) - 4x[3]*x[7] - 3x[43]*x[55]) - 4x[57] - 5x[16]*x[46]) + 4x[43]*x[52] + 3x[2]*x[16]*x[24]*x[39]*x[44]*x[58]  …  (-5x[23]*x[37] - 5x[13]*x[40])*(4x[43]*x[52] + 3x[2]*x[16]*x[24]*x[39]*x[44]*x[58]) + (4x[3]*x[31]*x[34] + 3x[16]*x[24]*x[25]*x[30]*x[44]*x[58])*((-4x[3]*x[38]*x[57] - 2x[3

In [249]:
Vs=simplify.(expand.(V))


LoadError: Failed to apply rule ~(z::_isone) * ~x => ~x on expression x[8]*x[20]

In [250]:

file=open("signature.txt","w")
for i in modify_matrix_coef(Vs,6)
    write(file,string(i))
    write(file,"\n")
end
close(file)

println(filesize("signature.txt"))

7422


# Size of public key, private key, and signature vector

In [263]:
filesizes = []
public_keys = []
private_keys = []
for i in 1:50
    A, B, C, D = matrix_kl(5, 3, 64, 2, 3, 6)

    file = open("publicKey.txt", "w")
    for i in 1:Int(size(C, 1))
        for j in 1:Int(size(C, 2))
            write(file, string(C[i, j]))
            write(file, " ")
        end
        write(file, "\n")
    end
    close(file)
    append!(public_keys, filesize("publicKey.txt"))

    file = open("privateKey.txt", "w")
    for i in 1:Int(size(D, 1))
        for j in 1:Int(size(D, 2))
            write(file, string(D[i, j]))
            write(file, " ")
        end
        write(file, "\n")
    end
    close(file)
    append!(private_keys, filesize("privateKey.txt"))

    
    
    U, Un = polys(message)
    V = Un * D
    Vs=simplify.(expand.(V))
    
    file = open("signature.txt", "w")
    for i in V
        write(file, string(i))
        write(file, "\n")
    end
    close(file)
    println(filesize("signature.txt"))
    
    file=open("signature.txt","w")
    for i in modify_matrix_coef(Vs,6)
        write(file,string(i))
        write(file,"\n")
    end
    close(file)
    append!(filesizes, filesize("signature.txt"))
    

end
println(filesizes)
println(mean(filesizes))

println(public_keys)
println(mean(public_keys))

println(private_keys)
println(mean(private_keys))

2365
3289
5357
5331
6372
3745
5342
3421
4038
2495
6164
6836
6290
2282
5595
5884
5562
7944
6003
5566
3356
2426
3522
6338
7821
5549
5748
2486
6157
5271
2486
3580
7536
6067
4259
3313
4011
3884
6324
3878
6576
3551
5175
6268
4261
5573
4875
3886
6438
2418
Any[4437, 17906, 45165, 27231, 32790, 18109, 31044, 18102, 18643, 6042, 46776, 31328, 49498, 5984, 35326, 47028, 16743, 65994, 40792, 59248, 15619, 8388, 20824, 43628, 52872, 39788, 18025, 9618, 67288, 33368, 9567, 19342, 49066, 38821, 19291, 15970, 26597, 21487, 33336, 22315, 56787, 18953, 37694, 41896, 15845, 38392, 41052, 14049, 48347, 7188]
30071.98
Any[4646, 3781, 3693, 3789, 4083, 4519, 3935, 4631, 4313, 5059, 3848, 2918, 4039, 4879, 4099, 3709, 4161, 3767, 4343, 3910, 5245, 4756, 4209, 3974, 3481, 3902, 3862, 4826, 3695, 4330, 5081, 4797, 3854, 3755, 4476, 4443, 4229, 4313, 3728, 4076, 3534, 4420, 4153, 3872, 4230, 4067, 3881, 4222, 3785, 4397]
4154.3
Any[1296, 2223, 4288, 4262, 5295, 2676, 4273, 2352, 2961, 1426, 5087, 5759, 5221, 1

In [227]:
substitution_dict = Dict()
for i in 1:64
    substitution_dict[x[i]] = rand(0:1)
end
substitution_dict

Dict{Any, Any} with 64 entries:
  x[31] => 0
  x[43] => 0
  x[36] => 1
  x[6]  => 1
  x[37] => 1
  x[49] => 1
  x[46] => 0
  x[56] => 0
  x[16] => 1
  x[57] => 0
  x[5]  => 0
  x[42] => 0
  x[60] => 1
  x[2]  => 0
  x[47] => 0
  x[61] => 1
  x[38] => 0
  x[50] => 0
  x[29] => 0
  x[8]  => 1
  x[28] => 0
  x[53] => 0
  x[13] => 1
  x[33] => 0
  x[11] => 1
  ⋮     => ⋮

In [233]:
W = V * C

1×3 Matrix{Num}:
 (3x[20]*x[33]*x[47] + 2x[30]*x[32]*x[40])*((-4x[8]*x[31]*x[46] - 5x[15]*x[20]*x[42])*(4x[43]*x[52] + 3x[2]*x[16]*x[24]*x[39]*x[44]*x[58]) + (3x[16]*x[25]*x[35]*x[43]*x[44]*x[58] + 4x[3]*x[9]*x[20]*x[29]*x[31]*x[34]*x[43]*x[52]*x[57])*(1.0 + (-4x[10] - 2x[5]*x[45])*((-x[1]*x[9] - 5x[10]*x[45])*(-3x[43]*x[46] - 2x[11]*x[19]*x[24]) - 2x[39] - 5x[22]*x[26]*x[33]) + (-2x[38] - x[1]*x[6]*x[24])*(-3x[43]*x[46] - 2x[11]*x[19]*x[24]) + (-5(x[22]^2)*x[17] - 4x[2]*x[37]*x[48])*((-x[7] - 5x[1]*x[13])*((-x[1]*x[9] - 5x[10]*x[45])*(-3x[43]*x[46] - 2x[11]*x[19]*x[24]) - 2x[39] - 5x[22]*x[26]*x[33]) + (-3x[43]*x[46] - 2x[11]*x[19]*x[24])*(-x[10] - 3x[12]*x[23]*x[50]) - 5x[5]*x[16]*x[22] - 5x[20]*x[23]*x[43]) + (-4x[8]*x[31]*x[46] - 5x[15]*x[20]*x[42])*((-5x[19] - 3x[11]*x[20]*x[38])*((-x[7] - 5x[1]*x[13])*((-x[1]*x[9] - 5x[10]*x[45])*(-3x[43]*x[46] - 2x[11]*x[19]*x[24]) - 2x[39] - 5x[22]*x[26]*x[33]) + (-3x[43]*x[46] - 2x[11]*x[19]*x[24])*(-x[10] - 3x[12]*x[23]*x[50]) - 5x[5]*x[16]*x

In [234]:
substitute.(W, (substitution_dict,))

1×3 Matrix{Num}:
 0.0  0.0  0.0

# Verification

In [331]:
A, B, C, D = matrix_kl(5, 3, 64, 2, 3, 6)
U, Un = polys(message)
V = Un * D

1×5 Matrix{Num}:
 (-x[22] - 5x[31]*x[43])*(4x[43]*x[52] + 3x[2]*x[16]*x[24]*x[39]*x[44]*x[58]) + (4x[3]*x[31]*x[34] + 3x[16]*x[24]*x[25]*x[30]*x[44]*x[58])*((-x[22]*x[48] - x[58]*x[62])*(-x[22] - 5x[31]*x[43]) - 2x[45] - 5x[49]) + (3x[16]*x[25]*x[35]*x[43]*x[44]*x[58] + 4x[3]*x[9]*x[20]*x[29]*x[31]*x[34]*x[43]*x[52]*x[57])*((-x[22] - 5x[31]*x[43])*((-3x[17]*x[23] - 5x[4]*x[8]*x[61])*(-x[22]*x[48] - x[58]*x[62]) - x[28] - 2x[31]*x[53]*x[62]) + (-2x[45] - 5x[49])*(-3x[17]*x[23] - 5x[4]*x[8]*x[61]) - x[2]*x[24]*x[45] - 2x[1]*x[14]*x[59])  …  (4x[3]*x[31]*x[34] + 3x[16]*x[24]*x[25]*x[30]*x[44]*x[58])*((-x[22]*x[48] - x[58]*x[62])*(-4x[21]*x[27] - 5x[32]*x[46]) + (-5x[15]*x[35]*x[43] - 2x[24]*x[54]*x[57])*((-x[22]*x[48] - x[58]*x[62])*(-x[22] - 5x[31]*x[43]) - 2x[45] - 5x[49]) - 3(x[6]^2)*x[60] - 4x[20]*x[55]) + (4x[43]*x[52] + 3x[2]*x[16]*x[24]*x[39]*x[44]*x[58])*((-x[22] - 5x[31]*x[43])*(-5x[15]*x[35]*x[43] - 2x[24]*x[54]*x[57]) - 4x[21]*x[27] - 5x[32]*x[46]) + (3x[16]*x[25]*x[35]*x[43]*x

In [332]:
function verification(C,V,Un)
#     cnt=0
#     for i in 1:100
#         substitution_dict = Dict()
#         for i in 1:64
#             substitution_dict[x[i]] = rand(0:1)
#         end

#         W=V*C
#         for i in 1:2
#             i=rand(1:3)
#             left=substitute.(W[1,i], (substitution_dict,))
#             right=substitute.(Un[1,i], (substitution_dict,))

#             if isequal(left,right)
#                 cnt=cnt+1
#             end
#         end
#     end
#     return (cnt==200)
    cnt=0
    for i in 1:20
        substitution_dict = Dict()
        for i in 1:64
            substitution_dict[x[i]] = rand(0:1)
        end
        
        W = V * C
        left=substitute.(W, (substitution_dict,))
        right=substitute.(Un, (substitution_dict,))
        if isequal(left,right)
            cnt=cnt+1
        end
    end
    return println(cnt==20)
    
    
#     W=V*C
#     i=rand(1:5)
#     Wi=simplify(expand(W[1,i]))
#     if isequal(Wi,simplify(expand(Un[1,i])))
#             println("good")
#             println(Un[1,i])
#             println(Wi)
#         end

end

@time verification(C,V,Un)

true
  0.211834 seconds (1.27 M allocations: 57.003 MiB, 15.61% compilation time)


In [333]:
for i in 1:100
    @time verification(C,V,Un)
end

true
  0.170951 seconds (1.20 M allocations: 52.616 MiB)
true
  0.175531 seconds (1.20 M allocations: 52.624 MiB)
true
  0.232069 seconds (1.20 M allocations: 52.624 MiB, 26.66% gc time)
true
  0.168339 seconds (1.20 M allocations: 52.625 MiB)
true
  0.168074 seconds (1.20 M allocations: 52.624 MiB)
true
  0.166027 seconds (1.20 M allocations: 52.625 MiB)
true
  0.166360 seconds (1.20 M allocations: 52.625 MiB)
true
  0.174281 seconds (1.20 M allocations: 52.624 MiB)
true
  0.210677 seconds (1.20 M allocations: 52.625 MiB, 16.19% gc time)
true
  0.172824 seconds (1.20 M allocations: 52.625 MiB)
true
  0.174351 seconds (1.20 M allocations: 52.624 MiB)
true
  0.172459 seconds (1.20 M allocations: 52.624 MiB)
true
  0.177409 seconds (1.20 M allocations: 52.625 MiB)
true
  0.168525 seconds (1.20 M allocations: 52.624 MiB)
true
  0.205698 seconds (1.20 M allocations: 52.624 MiB, 15.70% gc time)
true
  0.174629 seconds (1.20 M allocations: 52.625 MiB)
true
  0.166003 seconds (1.20 M allocati